In [1]:
!nvidia-smi

Tue Sep 17 13:22:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Overwriting exemplo1.cu


In [4]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [5]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [6]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [7]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [8]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


In [9]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Writing stocks.cu


In [10]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [12]:
!./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00156019
Tempo de CÓPIA (em segundos)  0.201285


**Exercício 1**

In [14]:
%%writefile exercicio1.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Writing exercicio1.cu


In [15]:
!nvcc -arch=sm_70 -std=c++14 exercicio1.cu -o exercicio1

In [16]:
!./exercicio1 < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00223852
Tempo de CÓPIA (em segundos)  0.200138


**Exercício 2**

In [43]:
%%writefile exercicio2.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main(){

  string nome_arquivo;
  cin >> nome_arquivo;

  cout << nome_arquivo;

  ifstream entrada(nome_arquivo);

  if (!entrada.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

  thrust::host_vector<double> host;

  double preco_medio_10_anos = 0;


  string linha;

  int n = 0;

  while (getline(entrada, linha)) {
        try {
            if (!linha.empty()) {
                double valor = stod(linha);
                host.push_back(valor);
                preco_medio_10_anos = preco_medio_10_anos + valor;
                n = n +1;
            }
        } catch (const std::invalid_argument& e) {
            cerr << "Linha inválida encontrada: '" << linha << "'" << endl;
        }
  }

  entrada.close();

  preco_medio_10_anos = preco_medio_10_anos/n;

  cout << "Preço médio nos ultimos 10 anos: " << preco_medio_10_anos << endl;

  double preco_medio_365_dias = 0;

  double maior_preco = host[0];
  double menor_preco = host[0];

  for (int i = n - 1; i >= n - 365 && i >= 0; i--){
    preco_medio_365_dias = preco_medio_365_dias + host[i];
    if (host[i] < menor_preco){
      menor_preco = host[i];
    }
    if (host[i] > maior_preco){
      maior_preco = host[i];
    }
  }

  preco_medio_365_dias = preco_medio_365_dias/365;


  cout << "Preço médio nos ultimos 365 dias: " << preco_medio_365_dias << endl;

  cout << "Maior preço nos últimos 365 dias: " << maior_preco << endl;

  cout << "Menor preço nos últimos 365 dias: " << menor_preco << endl;

  return 0;

}

Overwriting exercicio2.cu


In [44]:
!nvcc -arch=sm_70 -std=c++14 exercicio2.cu -o exercicio2

In [45]:
!./exercicio2

stocks-google.txt
'
'
'
'
'
'
'
Preço médio nos ultimos 10 anos: 1580.08
Preço médio nos ultimos 365 dias: 1615.81
Maior preço nos últimos 365 dias: 2200
Menor preço nos últimos 365 dias: 1015.24


**Exercício 3**

In [48]:
%%writefile exercicio3.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <sstream>                  // Biblioteca para manipulação de fluxos de string
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main(){

  string nome_arquivo;
  cin >> nome_arquivo;

  cout << nome_arquivo;

  ifstream entrada(nome_arquivo);

  if (!entrada.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

  thrust::device_vector<double> apple;

  thrust::device_vector<double> microsoft;

  thrust::device_vector<double> diferenca;


  string linha;

  int n = 0;

  while (getline(entrada, linha)) {
        try {
            if (!linha.empty()) {

                stringstream ss(linha);
                string parte1, parte2;

                getline(ss, parte1, ',');
                getline(ss, parte2, ',');

                double valor1 = stod(parte1);
                double valor2 = stod(parte2);
                apple.push_back(valor1);
                microsoft.push_back(valor2);

                double valor_diferenca;
                valor_diferenca = valor1 - valor2;

                diferenca.push_back(valor_diferenca);

                n = n + 1;

            }
        } catch (const std::invalid_argument& e) {
            cerr << "Linha inválida encontrada: '" << linha << "'" << endl;
        }
  }

  entrada.close();

  double preco_medio_diferenca = 0;

  for (int i = 0; i < n; i++){

    preco_medio_diferenca = preco_medio_diferenca + diferenca[i];

  }

  preco_medio_diferenca = preco_medio_diferenca/n;


  cout << "Preço médio das diferenças: " << preco_medio_diferenca << endl;

  return 0;

}

Overwriting exercicio3.cu


In [49]:
!nvcc -arch=sm_70 -std=c++14 exercicio3.cu -o exercicio3

In [50]:
!./exercicio3

stocks2.txt
stocks2.txtPreço médio das diferenças: -265.232
